In [ ]:
import pandas as pd
data= pd.read_csv('/kaggle/input/friday-csv/friday.csv')
data.head()

In [ ]:
import pandas as pd
import numpy as np

# Extract required features with exact column names
features = data[['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Label']]

# Convert IP addresses to numeric format
features['Src IP'] = features['Src IP'].apply(lambda x: int(x.replace('.', '')))
features['Dst IP'] = features['Dst IP'].apply(lambda x: int(x.replace('.', '')))

#Protocol were already lableled 0 for unknows, 1 for ICMP, 6 for TCP, 17 for UDP

# Create binary labels (modify according to your label values)
# 'BENIGN' is normal traffic and everything else is an attack
features['Label'] = (features['Label'] != 'BENIGN').astype(int)

# Convert to numpy arrays for further use
X = features[['Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol']].values.astype(np.float32)
y = features['Label'].values.astype(np.float32)

# Print shapes to verify
print("Features shape:", X.shape)
print("Labels shape:", y.shape)

# Print first few rows to verify preprocessing
print("\nFirst few preprocessed rows:")
print("Features:")
print(X[:5])
print("\nLabels:")
print(y[:5])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Custom Dataset class
class NetworkTrafficDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Neural Network Model
class IntrusionDetectionModel(nn.Module):
    def __init__(self):
        super(IntrusionDetectionModel, self).__init__()
        self.layer1 = nn.Linear(5, 64)  # 5 input features
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.layer3(x))
        return x

# Training function
def train_model(X, y, epochs=5, batch_size=32):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create datasets
    train_dataset = NetworkTrafficDataset(X_train, y_train)
    test_dataset = NetworkTrafficDataset(X_test, y_test)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Initialize model, loss, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = IntrusionDetectionModel().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        # Print statistics
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')
        
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Validation Accuracy: {accuracy:.2f}%')
    
    return model

# Main execution
if __name__ == "__main__":
    # Assuming X and y are your preprocessed data from earlier
    # Train the model
    model = train_model(X, y)
    
    # 3. Save the model for later use
    torch.save(model.state_dict(), 'intrusion_detection_model.pth')
    
    # Save the model architecture and weights together (alternative method)
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_architecture': IntrusionDetectionModel(),
    }, 'intrusion_detection_full_model.pt')